In [5]:
import osmnx as ox
import networkx as nx
from heapq import heappop, heappush
from math import sqrt, atan2, degrees
import ipyleaflet
from ipyleaflet import Map, Marker, Polyline, LayerGroup, FullScreenControl, basemaps
import ipywidgets as widgets
from IPython.display import display
import time
import threading

# Define the area of interest
place = 'Manhattan, New York, USA'
G = ox.graph_from_place(place, network_type='drive')

# Create an ipyleaflet map for interactive pin placement and visualization
m_ipy = Map(center=(40.748817, -73.985428), zoom=13, basemap=basemaps.CartoDB.DarkMatter)
m_ipy.add_control(FullScreenControl())

# Initialize variables to store start and end coordinates
start_coords = None
end_coords = None
markers = LayerGroup(name='Markers')
m_ipy.add_layer(markers)

# Event handler for map clicks
def handle_map_click(**kwargs):
    global start_coords, end_coords
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        if start_coords is None:
            start_coords = coords
            marker = Marker(location=start_coords, draggable=False, title='Start')
            markers.add_layer(marker)
        elif end_coords is None:
            end_coords = coords
            marker = Marker(location=end_coords, draggable=False, title='End')
            markers.add_layer(marker)
        else:
            print("Both start and end coordinates are set")

m_ipy.on_interaction(handle_map_click)

display(m_ipy)

# Define the Euclidean heuristic function
def euclidean_heuristic(node1, node2):
    lat1, lon1 = G.nodes[node1]['y'], G.nodes[node1]['x']
    lat2, lon2 = G.nodes[node2]['y'], G.nodes[node2]['x']
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

# Calculate the angle between two points
def calculate_angle(point1, point2):
    return degrees(atan2(point2[1] - point1[1], point2[0] - point1[0]))

# Check if a node is within the allowed angle range
def is_within_angle_range(current, neighbor, target, allowed_angle=60):
    current_coords = (G.nodes[current]['x'], G.nodes[current]['y'])
    neighbor_coords = (G.nodes[neighbor]['x'], G.nodes[neighbor]['y'])
    target_coords = (G.nodes[target]['x'], G.nodes[target]['y'])
    
    angle_to_target = calculate_angle(current_coords, target_coords)
    angle_to_neighbor = calculate_angle(current_coords, neighbor_coords)
    
    angle_diff = abs(angle_to_neighbor - angle_to_target)
    return min(angle_diff, 360 - angle_diff) <= allowed_angle

# Bidirectional A* search algorithm with parallel search and directional constraint
def bidirectional_a_star_search(G, start, end):
    explored_edges = []
    meeting_node = None
    search_complete = threading.Event()
    lock = threading.Lock()

    def search_direction(open_set, open_set_hash, came_from, g_score, f_score, other_set_hash, direction):
        nonlocal meeting_node
        
        while open_set and not search_complete.is_set():
            current = heappop(open_set)[1]
            open_set_hash.remove(current)

            with lock:
                if current in other_set_hash:
                    meeting_node = current
                    search_complete.set()
                    return

            for neighbor in G.neighbors(current):
                if search_complete.is_set():
                    return
                if not is_within_angle_range(current, neighbor, end if direction == 'forward' else start):
                    continue
                
                tentative_g_score = g_score[current] + G[current][neighbor][0]['length']
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = g_score[neighbor] + euclidean_heuristic(neighbor, end if direction == 'forward' else start)
                    if neighbor not in open_set_hash:
                        heappush(open_set, (f_score[neighbor], neighbor))
                        open_set_hash.add(neighbor)
                        with lock:
                            explored_edges.append((current, neighbor))

    # Initialize forward search
    open_set_start = [(0, start)]
    came_from_start = {}
    g_score_start = {start: 0}
    f_score_start = {start: euclidean_heuristic(start, end)}
    open_set_hash_start = {start}

    # Initialize backward search
    open_set_end = [(0, end)]
    came_from_end = {}
    g_score_end = {end: 0}
    f_score_end = {end: euclidean_heuristic(end, start)}
    open_set_hash_end = {end}

    # Start search in parallel using threads
    thread_forward = threading.Thread(target=search_direction, args=(open_set_start, open_set_hash_start, came_from_start, g_score_start, f_score_start, open_set_hash_end, 'forward'))
    thread_backward = threading.Thread(target=search_direction, args=(open_set_end, open_set_hash_end, came_from_end, g_score_end, f_score_end, open_set_hash_start, 'backward'))

    thread_forward.start()
    thread_backward.start()

    thread_forward.join()
    thread_backward.join()

    if meeting_node:
        path_start_to_meeting = reconstruct_path(came_from_start, meeting_node)
        path_meeting_to_end = reconstruct_path(came_from_end, meeting_node)[::-1]
        path = path_start_to_meeting + path_meeting_to_end[1:]
        return path, explored_edges, True

    return None, explored_edges, False

def animate_bidirectional_a_star():
    global start_coords, end_coords
    assert start_coords is not None and end_coords is not None, "Set start and end coordinates on the map."
    start_node = ox.distance.nearest_nodes(G, start_coords[1], start_coords[0])
    end_node = ox.distance.nearest_nodes(G, end_coords[1], end_coords[0])

    explored_layer = LayerGroup()
    path_layer = LayerGroup()
    m_ipy.add_layer(explored_layer)
    m_ipy.add_layer(path_layer)

    # Run bidirectional A* search and handle results
    path, new_edges, is_final = bidirectional_a_star_search(G, start_node, end_node)
    if path is None:
        print("No path found")
        return

    # Animate explored edges
    for edge in new_edges:
        line = Polyline(
            locations=[
                (G.nodes[edge[0]]['y'], G.nodes[edge[0]]['x']),
                (G.nodes[edge[1]]['y'], G.nodes[edge[1]]['x'])
            ],
            color='blue',
            opacity=0.5
        )
        explored_layer.add_layer(line)
        time.sleep(0.01)  # Adjust delay for animation speed

    # Display the final path
    if path:
        path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]
        final_polyline = Polyline(locations=path_coords, color='yellow', weight=5)
        path_layer.add_layer(final_polyline)

        # Calculate path length
        path_length = sum(G[path[i]][path[i+1]][0]['length'] for i in range(len(path)-1))
        print(f'Path Length: {path_length:.2f} meters')

    time.sleep(0.1)  # Adjust delay for animation speed

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]

# Button to trigger Bidirectional A* visualization
button = widgets.Button(description="Run Animated Bidirectional A* Search")
button.on_click(lambda x: animate_bidirectional_a_star())
display(button)

Map(center=[40.748817, -73.985428], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Button(description='Run Animated Bidirectional A* Search', style=ButtonStyle())

Path Length: 7523.26 meters
Path Length: 7523.26 meters
